In [711]:
import pandas as pd
import numpy as np

In [712]:
db = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\63 сш.xlsx").map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

In [713]:
to_read = {
    1 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Абай облысы (Done)\Done COPY - Қожагелді батыр мектебі 4,8 сын БЖБ ТЖБ.xlsx",
    2 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Абай облысы (Done)\Done COPY - сш Маяковского СОР и Соч  1 четв.xlsx",
    3 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Акмолинская область (1 remaining)\Done COPY - 4-8 И Құтпанұлы Аршалы ауданы Жібек жолы ауылы.xlsx",
    4 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Акмолинская область (1 remaining)\Done COPY - Ж. Бектұров атындағы №3 ЖОББМ 4-8сынып.xlsx",
    5 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Акмолинская область (1 remaining)\Done COPY - ШОртанды №3 ЖОББМ БЖБ және ТЖБ қорытындысы (1).xlsx",
    6 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - 4,8  сыныптар    СОР и СОЧ  ХРОМТАУ.xlsx", # Актюбинская область
    7 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY №2 Бадамша ОМ СОР и СОЧ.xlsx", # Актюбинская область
    8 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - №4 ШЖОББМ.xlsx", # Актюбинская область
    9 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - Ақтөбе об., Қобда ауд.,Қобда қазақ орта мектебі.xlsx", # Актюбинская область
    10 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - Байғанин ауданы. Т.Жармағамбетов атындағы орта мектебі.xlsx", # Актюбинская область
    11 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - СОР и СОЧ . СШ№3 г.Шалкар.xlsx", # Актюбинская область
    12 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - СОР и СОЧ 1ч. 4 и 8 классы Мартук.xlsx", # Актюбинская область
    13 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - Ырғыз ауданы, Ы.Алтынсарин ат ЖББОМ СОР СОЧ  4,8 класс.xlsx", # Актюбинская область
    14 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\REVISE! COPY - Мугалжар ауданы, Кандыагаш 3 мектеп-гимназиясы 4,8 класс.xlsx", # Актюбинская область
    15 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\REVISE! COPY - СОР и СОЧ Айтеке би Т.Жургенов.xlsx", # Актюбинская область
    16 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\REVISE! COPY - СШ им Жусибалиева.xlsx", # Актюбинская область
    17 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Алматы облысы\Done COPY - Алматы, Баканас 4-8 сынып СОР, СОЧ.xlsx", # Алматинская область
    18 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Алматы облысы\Done COPY - Нарынқол СОР и СОЧ.xlsx", # Алматинская область
    19 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Алматы облысы\REVISE! Done COPY - Ұйғыр ауданы Шонжы а Абай атындағы МГ.xlsx", # Алматинская область
    20 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Атырауская область\Done COPY - «Атырау облысы Білім беру басқармасының Индер ауданды...қін...тындағы орта мектеп» коммуналдық мемлекеттік мекемесі.xlsx", # Атырауская область
    21 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Атырауская область\Done COPY - СОР и СОЧ  Х.Досмұхамедұлы.xlsx", # Атырауская область
    22 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\ВКО\NO MAX COPY - СОР, СОЧ СШ М.Ауэзова ВКО г.Зайсан - Copy.xlsx", # ВКО
    23 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\ВКО\NO MAX COPY - ШҚО, Тарбағатай ауданы, Абай мектебі - Copy.xlsx", # ВКО
    24 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\ВКО\NO MAX in 4 grade COPY - Ж.Болғанбаев ОМЛ СОР и СОЧ.xlsx", # ВКО
    25 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\ВКО\NO MAX sor COPY - БЖБ ТЖБ №5 ОМ Шемонаиха.xlsx", # ВКО
    26 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Жамбылская область\COPY - 4-8_сынып_БЖБ-ТЖБ Ә.Бөкейхан атындағы  №15 тірек мектеп-лицей.xlsx", # Жамбылская область
    27 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Жамбылская область\COPY 2 - Мойынкум 4 -8  Мойынқұм ауданы, Т.Рысқұлов тірек.xlsx", # Жамбылская область
    28 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Жетисуская область\COPY - Ескелдинский район школа Шестакова.xlsx", # Жетисуская область
    29 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Жетисуская область\COPY - Жетісу облысы Сарқан қаласы Абай атындағы мектеп-гимназия.xlsx", # Жетисуская область
    30 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Жетисуская область\COPY - СОР и СОЧ -шаблон Когалинская СШ2.xlsx", # Жетисуская область
    31 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\ЗКО\COPY - Жымпиты СОР, СОЧ 1 Тоқсан.xlsx", # ЗКО
    32 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\ЗКО\COPY - М-С.Бабажанов.xlsx", # ЗКО
    33 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\ЗКО\NO MAX COPY - СОР и СОЧ -ЗКО Жанибекский район.xlsx", # ЗКО
    34 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Костанайская область\COPY - СОР СОЧ Рудный қаласы білім бөлімінің Бейімбет Майлин атындағы №7 мектеп-гимназиясы.xlsx", # Костанайская область
    35 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Костанайская область\COPY - Узункольская ООШ 1 СОР-СОЧ.xlsx", # Костанайская область
    36 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Костанайская область\NO MAX COPY - ШГ им.А.Кунанбаева г.Аркалык СОР и СОЧ.xlsx", # Костанайская область
    37 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Костанайская область\NO MAX COPY - ШЛ имени Абая сор и соч.xlsx", # Костанайская область
    38 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Кызылординская область\COPY - СОР и СОЧ -шаблон №250 мектеп Қармақшы.xlsx", # Кызылординская область
    39 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Кызылординская область\COPY (exception) - 286 орта мектеп 8-4.xlsx", # Кызылординская область !!!!!!!!!!!!!!!!!!!!!!!!
    40 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Мангистауская область\COPY - Маңғыстау облысы Қарақия ауданы Құрық ауылы  №6 ЖББМ.xlsx 4,8-сынып.xlsx", # Мангистайская область
    41 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Мангистауская область\COPY - Маңғыстау облысы Маңғыстау ауданы Шетпе ауылы  Н.Жұбаев атындағы ЖББМ 4,8-сынып.xlsx", # Мангистайская область
    42 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Аққулы СОР и СОЧ.xlsx", # Павлодарская область !!!!!!!!!!!!!!!!!!!!!!!
    43 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Баянаул, З.Акишев СОР и СОЧ.xlsx", # Павлодарская область !!!!!!!!!!!!!!!!!!!!!!!
    44 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Тереңкөл СОР, СОЧ-1.xlsx", # Павлодарская область !!!!!!!!!!!!!!!!!!!!!!!
    45 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Северо-Казахстанская область\COPY - КГУ ШГим ББ СОР СОЧ.xlsx", # Северо-Казахстанская область
    46 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Северо-Казахстанская область\COPY - СОР и СОЧ  Мамлютка.xlsx", # Северо-Казахстанская область
    47 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Северо-Казахстанская область\COPY - СОР и СОЧ -Қожаберген жырау ат. ЖОББМ үлгі.xlsx", # Северо-Казахстанская область
    48 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Северо-Казахстанская область\COPY (exception) - КГУ Явленская СШ№3 Есильский СКО (1).xlsx", # Северо-Казахстанская область !!!!!!!!!!!!!!!!!!!!!!!!!
    49 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Туркестанская область\COPY - А.Сулейменов СОР СОЧ талдау.xlsx", # Туркестанская область
    50 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Туркестанская область\COPY - САУРАН АУДАНЫ Ә.СЫҒАЙ ЖББМ 8 СЫНЫП СОР СОЧ.xlsx", # Туркестанская область
    51 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Туркестанская область\COPY - СОР и СОЧ .xlsx Амангелді Иманов ат.ЖББМ (1).xlsx", # Туркестанская область
    52 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Туркестанская область\COPY - СОР и СОЧ 8-4 сыныптар №7 ІТ мектеп-лицей (1).xlsx", # Туркестанская область
    53 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Туркестанская область\COPY - СОР и СОЧ 12 Б. Момышұлы  Кентау.xlsx", # Туркестанская область
    54 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Туркестанская область\COPY - 1Түркістан облысы, Бәйдібек ауданы С.Әбдіжаппаров ат ЖББМЖИ 4, 8 сыныптар  СОР и СОЧ.xlsx", # Туркестанская область
    55 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Улытауская область\NO MAX 8BIOL COPY - Б.Амалбеков 4,8 сыныптар БЖБ ТЖБ.xlsx", # Улытауская область
    56 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Карагандинская область\COPY - СОР и СОЧ - 1ч Осакаровка А.Бокейхана.xlsx", # Карагандинская область
    57 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Карагандинская область\COPY - СОР и СОЧ -  ОШ имени Абая Сарань.xlsx", # Карагандинская область
    58 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Карагандинская область\COPY - Қарағанды облысы Шет ауданы Ж. Ақылбаев МГ СОР и СОЧ -8 сыныптар.xlsx", # Карагандинская область
    59 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Карагандинская область\COPY - Теміртау қ Әлихан Бөкейхан атындағы гимназиясы 4-8 сыныптар СОР СОЧ 23.11.xlsx",
    60 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Карагандинская область\COPY - СОР и СОЧ ШГ.С.Сейфуллина.xlsx", # Карагандинская область
    61 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Карагандинская область\COPY - СОР и СОЧ 15.xlsx", # Карагандинская область
    62 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Карагандинская область\COPY - 4-8 классы ШЛ им Ш Уалиханова Абайский рн Карагандинская обл.xlsx" # Карагандинская область
}

In [714]:
# to_read = {42 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Павлодарская  область\COPY (exception) - Аққулы СОР и СОЧ.xlsx",}
df_dict = {}
ss_list = ['sor1', 'sor2', 'sor3', 'soch']
for key, file_path in to_read.items():
    try:
        df_dict[f"df{key}"] = pd.read_excel(file_path, sheet_name='data', dtype={col: str for col in ss_list})
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file: {file_path}. Error: {e}")

In [715]:
# Leaving only First anf Last names of each student
db['fio'] = db['fio'].map(lambda x: " ".join(str(x).split()[:2]) if len(str(x).split()) > 2 else x)

In [716]:
subject_map = {
    'английский язык' : 'ENFL',
    'ағылшын тілі' : 'ENFL',
    'шетел тілі' : 'ENFL',
    'биология' : 'BIOL',
    'география' : 'GEOG',
    'жаратылыстану' : 'SCIE',
    'казахский язык' : 'KZFL',
    'математика' : 'MATH',
    'алгебра' : 'MATH',
    'русский язык' : 'RUFL',
    'русский язык и литература' : 'RUFL',
    'орыс тілі' : 'RUFL',
    'физика' : 'PHYS',
    'химия' : 'CHEM',
    'қазақ тілі' : 'KZFL'
}

In [717]:
# df_dict['df59'][df_dict['df59']['fio'].str.contains('Шамған Ерасыл')]

In [718]:
df_dict['df42']['subject'].unique()

array(['биология', 'математика', 'физика', 'география', 'русский язык'],
      dtype=object)

In [719]:
noLang_names = set() # Storing student names whose language of study wasn't found
for key, df in df_dict.items():
    # Test
    must_have_cols = ['fio', 'subject', 'soch']
    for mhcol in must_have_cols:
        if not 'soch' in df.columns:
            print(f"! Warning ! : No '{mhcol}' column in '{key}'")
            break
    # Testing
    index_list = []
    for index, row in df.iterrows():
        if (pd.isna(df.loc[index, 'subject']) and pd.isna(df.loc[index, 'soch'])): # cheking for empty subject and soch values
            index_list.append(index)
    if len(index_list) > 0:
        # removing rows having with empty subject and soch values simultaneously
        df = df.drop(index=index_list).reset_index() # resetting index order back with step=1
            
    df = df.map(lambda x: " ".join(x.split()) if isinstance(x, str) else x) # Removing whitespaces
    df['fio'] = df['fio'].map(lambda x: " ".join(str(x).split()[:2]) if len(str(x).split()) > 2 else x) # Leaving only First anf Last names of each student

    # In some tables there is only one name for all subjects. Hence, we need to multiplicate the name to its corresponding subjects
    df['fio'] = df['fio'].ffill()
    # SWE Approach
    # -----------------------------------------
        # name = ''
        # for index, fio in enumerate(df['fio']):
        #     if isinstance(fio, str):
        #         name = fio
        #     else:
        #         df.loc[index, 'fio'] = name
    # -----------------------------------------
    
    # Sometimes, instead of a standard format (mixture of numbers and abbreviations), subjects are written in full words
    lang_db = db[['fio', 'lang']]
    for subject in df['subject'].unique():
        if (not ('4' in subject or '8' in subject)):
            # display(df[df['subject'].isna()])
            df['subject'] = df['subject'].str.lower()
            for index, subject in enumerate(df['subject']):
                if subject in subject_map:
                    df.loc[index, 'subject'] = subject_map[subject]
            # display(df)
            merged = pd.merge(df, lang_db, on='fio', how='left')
            merged['fio'] = merged['fio'].ffill()
            # display(merged)
            for index, lang in enumerate(merged['lang']):
                if (lang == 1) and (merged.loc[index, 'subject'] == 'RUFL'):
                    merged.loc[index, 'subject'] = 'RUSL'
                elif (lang == 2) and (merged.loc[index, 'subject'] == 'KZFL'):
                    merged.loc[index, 'subject'] = 'KZSL'
            
            merged = merged.drop_duplicates()

            for fio in merged['fio'].unique():
                table = merged[merged['fio'] == fio]
                table.loc[:, 'lang'] = table['lang'].ffill()
                merged.loc[merged['fio'] == fio, 'lang'] = table['lang'].unique()[0]
            
            naLangFio = merged[merged['lang'].isna()]['fio'].unique()
            noLang_names.update(name for name in naLangFio)
            break
    # df = merged
    # -----------------------------------------
    
    df = df.fillna('0')
    
    for col in ['sor1', 'sor2', 'sor3']:
        if col not in df.columns: 
            df[col] = 0

    df[ss_list] = df[ss_list].map(lambda x: ''.join(x.split()) if isinstance(x, str) else x)

    df.dropna(subset=['sor1', 'sor2', 'sor3', 'soch'], how='all', inplace=True)

    for index, char in enumerate(df['subject']):
        if ('4' in char or '8' in char):
            df.loc[index, 'grade'] = df.loc[index, 'subject'][0]
            df.loc[index, 'subject'] = df.loc[index, 'subject'][1:]
            
    df['grade'] = pd.to_numeric(df['grade'], errors='coerce')


    df[ss_list] = df[ss_list].map(lambda x: str(x).replace('\\', ' ').replace('/', ' ').replace('∕', ' ').replace('|', ' ').replace('-', ' ').replace('.', '').replace(',', ''))

    df = df.map(lambda x: np.nan if str(x).isspace() else np.nan if str(x) == '' else x)
    df.dropna(subset=df.columns, how='all', inplace=True)

    for col in ss_list:
        if ('sor1_max' not in df.columns) or ('sor2_max' not in df.columns) or ('sor3_max' not in df.columns) or ('soch_max' not in df.columns):
            df[f"{col}_max"] = df[col].apply(lambda x: str(x).split()[1] if (len(str(x).split()) > 1 and str(x) != '0') else np.nan)
        df[col] = df[col].apply(lambda x: str(x).split()[0] if len(str(x).split()) >= 1 else x)
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[f"{col}_max"] = pd.to_numeric(df[f"{col}_max"], errors='coerce')
    df = df.fillna(0)
    
    df = df.map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

    name = df.loc[2, 'fio']
    school = db.loc[db['fio'].str.contains(name), 'school_ru'].unique()
    
    # Test
    if len(school) == 1:
        df['school'] = school[0]
    else:
        print(f"!Attention! : '{key}' has no school name")

    df = df[~((df['sor1'] == 0) & (df['sor2'] == 0) & (df['sor3'] == 0))] # removing rows with no values
    
    reindexed_cols = ['school', 'fio', 'grade', 'subject', 'sor1', 'sor1_max', 'sor2', 'sor2_max', 'sor3', 'sor3_max', 'soch', 'soch_max']
    df = df.reindex(columns=reindexed_cols)

    # Swaping values if exam score is greater than exam max score
    df = df.reset_index(drop=True) # Resetting index for proper index iteration
    for col in ss_list:
        for index in range(len(df)):
            if (df.at[index, col] > df.at[index, f"{col}_max"] and df.at[index, f"{col}_max"] != 0):
                num = df.at[index, col]
                df.at[index, col] = df.at[index, f"{col}_max"]
                df.at[index, f"{col}_max"] = num

    df_dict[key] = df
                
    
# Test
for key, df in df_dict.items():
    df = df.drop(columns='school')
    if df.isna().values.any():
        print(f"Error: '{key}' contains NaN values")
        display(df[df.isna()].head(3))
    for subject in df['subject']:
        if len(subject) < 4:
            print(f"Error: 'subject' in '{key}' < 4 characters")
            display(df[df['subject'] == subject])
            break


!Attention! : 'df4' has no school name
!Attention! : 'df6' has no school name
!Attention! : 'df16' has no school name
!Attention! : 'df22' has no school name
!Attention! : 'df24' has no school name
!Attention! : 'df25' has no school name
!Attention! : 'df26' has no school name
!Attention! : 'df55' has no school name
!Attention! : 'df57' has no school name


In [720]:
for key, df in df_dict.items():
    for col in ss_list:
        t = df.iloc[0, [1, 2, 3]]
        t = t.to_frame().T
        display(t)
        # for index in range(len(df)):
            
        break
        # break
    break

,fio,grade,subject
0,Айтмұқаш Марғұлан,4,MATH


In [721]:
# Adding school names to the remaining dfs
schools_map = {
    'df4' : 'КГУ "Общеобразовательная школа №3 имени Жайыка Бектуров города Акколь"',
    'df6' : 'КГУ "Хромтауская средняя школа №1"',
    'df16' : 'КГУ "Уилская казахская средняя школа имени Ж.Жусибалиева"',
    'df22' : 'КГУ «Средняя школа имени М.Ауэзова»',
    'df24' : 'КГУ «Самарская средняя школа-лицей имени Ж.Болганбаева»',
    'df25' : 'КГУ «Комплекс общеобразовательная средняя школа-ясли-детский сад №5 имени Ахмета Байтурсынова»',
    'df26' : 'КГУ "Школа-лицей №15 имени Алихана Бокейхана отдела образования Жуалынского района управления образования акимата Жамбылской области"',
    'df55' : 'КГУ "Опорная школа (ресурсный центр) им. Б. Амалбекова"',
    'df57' : 'КГУ "Общеобразовательная школа имени Абая"'
}
for key, value in schools_map.items():
    for k, df in df_dict.items():
        if key == k:
            df.loc[:, 'school'] = schools_map[k]

In [722]:
# df_dict['df2'].head(10)

In [723]:
# print(len(noLang_names))
# noLang_names

In [724]:
# df_dict['df42'][df_dict['df42']['fio'] == 'Юлдашев Бахтжан']['subject'].unique()

In [725]:
# df_dict['df59'][df_dict['df59']['fio'].str.contains('Шамған Ерасыл')]

In [726]:
# for key, value in schools_map.items():
#     display(df_dict[key].head(3))

#### Removing rows where sor/soch > 0 and max == 0

In [727]:
# # Checking for DFs having exam score with no exam max score
# noMaxSet = set()
# for key, df in df_dict.items():
#     for col in ss_list:
#         for index in range(len(df)):
#             if (df.at[index, col] > 0) and (df.at[index, f"{col}_max"] == 0):
#                 noMaxSet.add(index)
#     df = df.drop(index=noMaxSet)
#     df = df.reset_index(drop=True)
#     df_dict[key] = df
# print(len(noMaxSet))
#                 # noMaxSet.add(f"No max: {to_read[int(key[2:])]}")
# # print(f"{len(noMaxSet)} school have no max:")
# # for path in noMaxSet:
# #     print(path)

---

#### Extracting dfs with no max scores

In [728]:
# Extracting dfs with zero max scores
noMaxList = []
for key, df in df_dict.items():
    if ((df['sor1_max'] == 0).all() and (df['sor1'] != 0).all()) or ((df['sor2_max'] == 0).all() and (df['sor2'] != 0).all()) or ((df['sor3_max'] == 0).all() and (df['sor3'] != 0).all()) or ((df['soch_max'] == 0).all() and (df['soch'] != 0).all()):
        noMaxList.append(key)
        path = str(key[2:])
        print(to_read[int(path)])
        display(df.head(3))
print(len(noMaxList))

C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY №2 Бадамша ОМ СОР и СОЧ.xlsx


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""Бадамшинская средняя школа №2""",Алдиярова Аружан,8,CHEM,10,0.0,10,0.0,8,0.0,14,0.0
1,"КГУ ""Бадамшинская средняя школа №2""",Алдиярова Аружан,8,BIOL,9,0.0,7,0.0,10,0.0,19,0.0
2,"КГУ ""Бадамшинская средняя школа №2""",Алдиярова Аружан,8,ENFL,6,0.0,10,0.0,0,0.0,14,0.0


C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\REVISE! COPY - СОР и СОЧ Айтеке би Т.Жургенов.xlsx


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""Общеобразовательная средняя школа имени Т...",Айділда Құдырет,4,KZFL,5,0.0,5,0.0,0,0.0,10.0,0.0
1,"КГУ ""Общеобразовательная средняя школа имени Т...",Айділда Құдырет,4,MATH,5,0.0,7,0.0,5,0.0,9.0,0.0
2,"КГУ ""Общеобразовательная средняя школа имени Т...",Айділда Құдырет,4,SCIE,4,0.0,5,0.0,0,0.0,6.0,0.0


C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\REVISE! COPY - СШ им Жусибалиева.xlsx


...

C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Северо-Казахстанская область\COPY - КГУ ШГим ББ СОР СОЧ.xlsx


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""Школа-гимназия имени Батыр Баяна""",Андреева Зарина,8,MATH,11,0.0,0,0.0,0,0.0,20.0,0.0
1,"КГУ ""Школа-гимназия имени Батыр Баяна""",Андреева Зарина,8,RUFL,10,0.0,8,0.0,0,0.0,17.0,0.0
2,"КГУ ""Школа-гимназия имени Батыр Баяна""",Андреева Зарина,8,KZSL,5,0.0,6,0.0,0,0.0,19.0,0.0


C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Северо-Казахстанская область\COPY - СОР и СОЧ  Мамлютка.xlsx


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""Мамлютская школа-гимназия №1""",Алдабергенова Самира,8,RUFL,12.0,0.0,13.0,0.0,0.0,0.0,23.0,0.0
1,"КГУ ""Мамлютская школа-гимназия №1""",Алдабергенова Самира,8,KZSL,6.0,0.0,4.0,0.0,0.0,0.0,21.0,0.0
2,"КГУ ""Мамлютская школа-гимназия №1""",Алдабергенова Самира,8,ENFL,9.0,0.0,8.0,0.0,0.0,0.0,16.0,0.0


11


In [729]:
# # Saving dfs with zero max scores to excel tables
# for noMax in noMaxList:
#     toExcel = df_dict[noMax]
#     schoolName = str(toExcel['school'].values[0])
#     if len(schoolName) > 3:
#         schoolName = schoolName.replace('"', ' ')
#         schoolName = " ".join(schoolName.split()) 
#         toExcel.to_excel(rf"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\noMaxSchools\{schoolName}.xlsx", index=False)
#     else:
#         toExcel.to_excel(rf"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\noMaxSchools\NoName.xlsx", index=False)

#### Checking if any DF has no school

In [730]:
for key, df in df_dict.items():
    if df['school'].isna().all():
        display(df)

---

#### Checking lang of the remaining 27 students

In [731]:
lang_df = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\Контингент, логин и пароль 13102023- — копия.xlsx", sheet_name='Лист2', dtype={'iin' : str}).map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

In [732]:
lang_df = lang_df[['iin', 'fio', 'lang']]

In [733]:
lang_df['fio'] = lang_df['fio'].apply(lambda x: " ".join(x.split()[:2]))

In [734]:
lang_df['lang'].unique()

array(['қазақ', 'орыс', 'өзбек'], dtype=object)

In [735]:
# 1 check
check1 = lang_df[lang_df['fio'].isin(noLang_names)]
display(check1)
foundLangNames = check1['fio']

,iin,fio,lang
1354,140828505432,Ермекбай Алан,қазақ
1392,130310602621,Сулейменова Диляра,қазақ
1420,131030503763,Абдыкарим Алдияр,қазақ
6959,140514505066,Амангелді Батырхан,қазақ
6997,100217554348,Каирбеков Айан,қазақ
8150,140824603001,Анас Сырғалым,қазақ


In [736]:
# # 2 check
# for lang in noLang_names:
#     to_display = lang_df[lang_df['fio'].str.contains(lang)]
#     if not to_display.empty:
#         display(to_display)

In [737]:
# Assigning lang to the three found
for key, df in df_dict.items():
    if df['fio'].isin(foundLangNames).any():
        df.loc[(df['fio'].isin(foundLangNames)) & (df['subject'] == 'RUFL'), 'subject'] = 'RUSL'


In [738]:
df_dict['df60']

,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""Школа - гимназия имени Сакена Сейфуллина""",Алиев Ануар,4,RUFL,3,13.0,5,11.0,0,0.0,11,18
1,"КГУ ""Школа - гимназия имени Сакена Сейфуллина""",Алиев Ануар,4,MATH,6,10.0,10,15.0,4,12.0,14,18
2,"КГУ ""Школа - гимназия имени Сакена Сейфуллина""",Алиев Ануар,4,SCIE,14,15.0,0,0.0,0,0.0,15,18
3,"КГУ ""Школа - гимназия имени Сакена Сейфуллина""",Арюкова Агата,4,RUFL,9,13.0,8,11.0,0,0.0,13,18
4,"КГУ ""Школа - гимназия имени Сакена Сейфуллина""",Арюкова Агата,4,MATH,6,10.0,8,15.0,7,12.0,14,18
...,...,...,...,...,...,...,...,...,...,...,...,...
328,"КГУ ""Школа - гимназия имени Сакена Сейфуллина""",Янбарцев Давид,8,ENFL,8,10.0,10,10.0,0,0.0,20,24
329,"КГУ ""Школа - гимназия имени Сакена Сейфуллина""",Янбарцев Давид,8,KZSL,8,10.0,13,15.0,0,0.0,26,30
330,"КГУ ""Школа - гимназия имени Сакена Сейфуллина""",Янбарцев Давид,8,MATH,12,13.0,0,0.0,0,0.0,15,20
331,"КГУ ""Школа - гимназия имени Сакена Сейфуллина""",Янбарцев Давид,8,PHYS,10,12.0,8,11.0,0,0.0,15,25


In [739]:
len(noLang_names)

37

---

#### Combining dfs to one table

In [740]:
result_df = pd.DataFrame()
for key, df in df_dict.items():
    if not key in noMaxList:
        result_df = pd.concat([result_df, df])
result_df = result_df.reset_index(drop=True)

In [741]:
result_df

,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,4,MATH,5.0,10.0,6.0,10.0,6.0,10.0,8.0,16.0
1,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,4,KZFL,6.0,10.0,5.0,10.0,0.0,0.0,9.0,16.0
2,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,4,SCIE,7.0,10.0,8.0,10.0,0.0,0.0,10.0,16.0
3,"КГУ ""СШ имени Кожагелды батыра""",Асанова Диана,4,MATH,7.0,10.0,8.0,10.0,8.0,10.0,8.0,16.0
4,"КГУ ""СШ имени Кожагелды батыра""",Асанова Диана,4,KZFL,7.0,10.0,7.0,10.0,0.0,0.0,12.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...
33868,"КГУ ""Школа-лицей имени Шокана Уалиханова""",Сайлаукен Әділ,8,MATH,7.0,15.0,0.0,0.0,0.0,0.0,11.0,20.0
33869,"КГУ ""Школа-лицей имени Шокана Уалиханова""",Сухарева Алиса,8,MATH,11.0,15.0,0.0,0.0,0.0,0.0,11.0,20.0
33870,"КГУ ""Школа-лицей имени Шокана Уалиханова""",Фризен Екатерина,8,MATH,9.0,15.0,0.0,0.0,0.0,0.0,17.0,20.0
33871,"КГУ ""Школа-лицей имени Шокана Уалиханова""",Хмелева Татьяна,8,MATH,8.0,15.0,0.0,0.0,0.0,0.0,9.0,20.0


In [742]:
# Some subject have a mixture of latin and russian characters
result_df.loc[result_df['subject'] == 'MATH мат', 'subject'] = 'MATH'
result_df.loc[result_df['subject'] == 'RUFL лит', 'subject'] = 'RUFL'
result_df.loc[result_df['subject'] == 'SCIE ест', 'subject'] = 'SCIE'
result_df.loc[result_df['subject'] == 'SCIL', 'subject'] = 'SCIE'
result_df.loc[result_df['subject'] == 'MATH', 'subject'] = 'MATH'
result_df.loc[result_df['subject'] == 'МАТН', 'subject'] = 'MATH'
result_df.loc[result_df['subject'] == 'KZFL', 'subject'] = 'KZFL'
result_df.loc[result_df['subject'] == 'КZFL', 'subject'] = 'KZFL'

In [743]:
chosenCols = ['MATH', 'KZFL', 'SCIE', 'RUFL', 'CHEM', 'BIOL', 'ENFL', 'PHYS', 'RUSL', 'KZSL']
result_df = result_df.loc[result_df['subject'].isin(chosenCols)]
result_df = result_df.reset_index(drop=True)

In [744]:
result_df['subject'].unique()

array(['MATH', 'KZFL', 'SCIE', 'RUFL', 'CHEM', 'BIOL', 'ENFL', 'PHYS',
       'RUSL', 'KZSL'], dtype=object)

In [745]:
for col in ss_list:
    for index in range(len(result_df)):
        if result_df.at[index, col] > result_df.at[index, f"{col}_max"]:
            display()

In [746]:
# # Swaping values if exam score is greater than exam max score
# for col in ss_list:
#     for index in range(len(result_df)):
#         if result_df.at[index, f"{col}"] > result_df.at[index, f"{col}_max"]:
#             num = result_df.at[index, f"{col}"]
#             result_df.at[index, f"{col}"] = result_df.at[index, f"{col}_max"]
#             result_df.at[index, f"{col}_max"] = num

In [747]:
result_df[result_df['fio'] == 'Тұрсынбек Рахымжан']['school'].unique()

array(['КГУ "Школа-лицей №15 имени Алихана Букейханова" отдела образования города Балхаш управления образования Карагандинской области'],
      dtype=object)

In [748]:
noMaxList

['df7',
 'df15',
 'df16',
 'df19',
 'df21',
 'df23',
 'df25',
 'df33',
 'df36',
 'df45',
 'df46']

In [749]:
result_df

,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,4,MATH,5.0,10.0,6.0,10.0,6.0,10.0,8.0,16.0
1,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,4,KZFL,6.0,10.0,5.0,10.0,0.0,0.0,9.0,16.0
2,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,4,SCIE,7.0,10.0,8.0,10.0,0.0,0.0,10.0,16.0
3,"КГУ ""СШ имени Кожагелды батыра""",Асанова Диана,4,MATH,7.0,10.0,8.0,10.0,8.0,10.0,8.0,16.0
4,"КГУ ""СШ имени Кожагелды батыра""",Асанова Диана,4,KZFL,7.0,10.0,7.0,10.0,0.0,0.0,12.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...
33796,"КГУ ""Школа-лицей имени Шокана Уалиханова""",Сайлаукен Әділ,8,MATH,7.0,15.0,0.0,0.0,0.0,0.0,11.0,20.0
33797,"КГУ ""Школа-лицей имени Шокана Уалиханова""",Сухарева Алиса,8,MATH,11.0,15.0,0.0,0.0,0.0,0.0,11.0,20.0
33798,"КГУ ""Школа-лицей имени Шокана Уалиханова""",Фризен Екатерина,8,MATH,9.0,15.0,0.0,0.0,0.0,0.0,17.0,20.0
33799,"КГУ ""Школа-лицей имени Шокана Уалиханова""",Хмелева Татьяна,8,MATH,8.0,15.0,0.0,0.0,0.0,0.0,9.0,20.0


#### Removing rows where sor/soch > 0 and max == 0

In [750]:
noMaxIndexSet = set()
for col in ss_list:
    for index in range(len(result_df)):
        if (result_df.at[index, col] > 0) and (result_df.at[index, f"{col}_max"] == 0):
            noMaxIndexSet.add(index)
result_df = result_df.drop(index=noMaxIndexSet).reset_index(drop=True)

In [751]:
len(noMaxIndexSet)

3698

In [752]:
# Double-check
doubleCheckSet = set()
for col in ss_list:
    for index in range(len(result_df)):
        if (result_df.at[index, col] > 0) and (result_df.at[index, f"{col}_max"] == 0):
            doubleCheckSet.add(index)
if len(doubleCheckSet) == 0:            
    print('All data is valid')
else:
    print(len(doubleCheckSet))

All data is valid


#### Checking for abnormal values

In [761]:
abValueSet = set()
for col in ss_list:
    for index in range(len(result_df)):
        if (result_df.at[index, col] > 50 or result_df.at[index, f"{col}_max"] > 50):
            abValueSet.add(index)
if len(abValueSet) == 0:
    print('All data is valid')
else:
    result_df = result_df.drop(index=abValueSet).reset_index(drop=True)        
    print(f"{len(abValueSet)} row removed")

12 row removed


In [753]:
print(len(df_dict.keys()))

62


In [754]:
result_df['school'].nunique()

50

In [758]:
result_df['fio'].nunique()

6090